<a href="https://colab.research.google.com/github/Vedant-Tibrewal/langchain-vector/blob/main/langchain101_google_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SETUP

In [15]:
from IPython.display import HTML, display

def set_css():
    display(HTML(''' <style> pre { white-space: pre-wrap; } </style> '''))

get_ipython().events.register('pre_run_cell', set_css)

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!pip install langchain deeplake openai tiktoken dotenv langchain-google-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.4/32.4 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.6/99.6 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 85.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.9 MB/s eta 0:00:00


In [5]:
from dotenv import load_dotenv

In [8]:
load_dotenv('/content/drive/MyDrive/rag/langchain-vector/langchain-vec.env')

True

## Google Search - Custom Search Engine

In [11]:
from langchain_core.tools import Tool
from langchain_google_community import GoogleSearchAPIWrapper

search = GoogleSearchAPIWrapper()

tool = Tool(
    name="google_search",
    description="Search Google for recent results.",
    func=search.run,
)

In [16]:
response = tool.run("Who launched Mario?")
print(response)

Dec 1, 2024 ... Shigeru Miyamoto, creator of the iconic Mario and Zelda video game franchises, has passed gas at the age of 72. r/tomorrow - Shigeru Miyamoto, ... Mario debuted as the player character of Donkey Kong, a 1981 platform game. Miyamoto created Mario because Nintendo was unable to license Popeye as the ... Sep 20, 2023 ... Mario Kart 64, back in the late 1990s. My parents got it for me after they bought me and my sister a Nintendo 64 with Super Mario 64, it was our second game. It is the successor to the 1983 arcade game Mario Bros. and the first game in the Super Mario series. Players control Mario, or his brother Luigi in the ... Mar 16, 2017 ... I know for the first year it was mostly Super Mario 64. I had Wave Race so I could play a game with friends. Then finally Mario Kart 64 came ... Jul 5, 2025 ... Mario Kart World was the biggest Nintendo Switch 2 launch games and while it seemed like a light lineup, it turns out Nintendo's racing game ... May 19, 2024 ... It was ei

### Number of Results

In [17]:
search = GoogleSearchAPIWrapper(k=1)

tool = Tool(
    name="I'm Feeling Lucky",
    description="Search Google and return the first result.",
    func=search.run,
)

In [20]:
response = tool.run("Who launched Mario?")
print(response)

Dec 1, 2024 ... Shigeru Miyamoto, creator of the iconic Mario and Zelda video game franchises, has passed gas at the age of 72. r/tomorrow - Shigeru Miyamoto, ...


## Metadata Results

In [21]:
search = GoogleSearchAPIWrapper()


def top5_results(query):
    return search.results(query, 5)


tool = Tool(
    name="Google Search Snippets",
    description="Search Google for recent results.",
    func=top5_results,
)

In [22]:
response = tool.run("Who launched Mario?")
print(response)

[{'title': 'Why people say Shigeru Miyamoto created Mario? : r/tomorrow', 'link': 'https://www.reddit.com/r/tomorrow/comments/1h3sxjp/why_people_say_shigeru_miyamoto_created_mario/', 'snippet': 'Dec 1, 2024 ... A heartwarming story: A young boy once showed his drawing of his favourite character, Super Mario, to the creator of Mario, Shigeru Miyamoto. In\xa0...'}, {'title': 'Mario - Wikipedia', 'link': 'https://en.wikipedia.org/wiki/Mario', 'snippet': 'Mario debuted as the player character of Donkey Kong, a 1981 platform game. Miyamoto created Mario because Nintendo was unable to license Popeye as the\xa0...'}, {'title': 'What was your first Mario Kart and what year did you play it? : r ...', 'link': 'https://www.reddit.com/r/mariokart/comments/16nj3y5/what_was_your_first_mario_kart_and_what_year_did/', 'snippet': 'Sep 20, 2023 ... Mario Kart 64, back in the late 1990s. My parents got it for me after they bought me and my sister a Nintendo 64 with Super Mario 64, it was our second game.

# Agent + Custom Search Engine

### Imports

In [23]:
from langchain.llms import OpenAI

from langchain.agents import AgentType
from langchain.agents import load_tools
from langchain.agents import initialize_agent

from langchain.agents import Tool
from langchain.utilities import GoogleSearchAPIWrapper

### Building Agent

In [24]:
llm = OpenAI(model="gpt-3.5-turbo-instruct", temperature=0)

/tmp/ipython-input-219042304.py:1: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm = OpenAI(model="gpt-3.5-turbo-instruct", temperature=0)


In [25]:
# remember to set the environment variables
# “GOOGLE_API_KEY” and “GOOGLE_CSE_ID” to be able to use
# Google Search via API.
search = GoogleSearchAPIWrapper()

/tmp/ipython-input-2583626623.py:4: LangChainDeprecationWarning: The class `GoogleSearchAPIWrapper` was deprecated in LangChain 0.0.33 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-google-community package and should be used instead. To use it run `pip install -U :class:`~langchain-google-community` and import as `from :class:`~langchain_google_community import GoogleSearchAPIWrapper``.
  search = GoogleSearchAPIWrapper()


In [26]:
tools = [
    Tool(
        name = "google-search",
        func=search.run,
        description="useful for when you need to search google to answer questions about current events"
    )
]

In [30]:
agent = initialize_agent(tools,
                         llm,
                         agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
                         verbose=True,
                         max_iterations=6)

In [31]:
response = agent("What's the latest news about US Tariff on India?")
print(response)



> Entering new AgentExecutor chain...
 I should search for news articles about US Tariff on India to get the latest information.
Action: google-search
Action Input: "US Tariff on India"
Observation: Jul 22, 2025 ... 'We're going to crush your economy': US Senator Lindsey Graham warns India, China, Brazil over 'cheap' Russian oil; threatens to 'tariff the ... Jul 17, 2025 ... Trump says India and US very close to finalising trade deal · Trump to hit India with 25% tariffs - plus 'penalty' for trade with Russia · Air ... 10 hours ago ... President Trump on Wednesday said that the United States will begin imposing 25% tariffs on goods imported from India on Aug. 1. 9 hours ago ... President Trump said Wednesday that goods from India will face a 25% tariff when entering the US beginning on Friday, a signal that ... 1 day ago ... On Friday, the United States is ready to impose a 26 percent tariff on all goods it imports from India. Dozens of countries are facing the same ... 13 hours ago 

In [32]:
response['output']

'The latest news about US Tariff on India is that the US is planning to impose a 25% tariff on goods imported from India starting on August 1st, but negotiations are still ongoing and a trade deal has not been reached yet.'

# Chaining Tools - Language Model + Google Search

### Imports

In [36]:
from langchain.llms import OpenAI
from langchain.agents import Tool
from langchain.utilities import GoogleSearchAPIWrapper
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.agents import initialize_agent, AgentType

In [37]:
llm = OpenAI(model="gpt-3.5-turbo-instruct", temperature=0)

prompt = PromptTemplate(
    input_variables=["query"],
    template="Write a summary of the following text: {query}"
)

summarize_chain = LLMChain(llm=llm, prompt=prompt)

/tmp/ipython-input-1638005415.py:8: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  summarize_chain = LLMChain(llm=llm, prompt=prompt)


In [38]:
# remember to set the environment variables
# “GOOGLE_API_KEY” and “GOOGLE_CSE_ID” to be able to use
# Google Search via API.
search = GoogleSearchAPIWrapper()

tools = [
    Tool(
        name="Search",
        func=search.run,
        description="useful for finding information about recent events"
    ),
    Tool(
       name='Summarizer',
       func=summarize_chain.run,
       description='useful for summarizing texts'
    )
]

In [39]:
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

In [40]:
response = agent("What's the latest news about the US tariffs on India? Then please summarize the results.")
print(response)



> Entering new AgentExecutor chain...
 I should use the Search tool to find information about recent events related to US tariffs on India.
Action: Search
Action Input: "US tariffs on India"
Observation: Jul 22, 2025 ... 'We're going to crush your economy': US Senator Lindsey Graham warns India, China, Brazil over 'cheap' Russian oil; threatens to 'tariff the ... Jul 17, 2025 ... Trump says India and US very close to finalising trade deal · Trump to hit India with 25% tariffs - plus 'penalty' for trade with Russia · Air ... 10 hours ago ... President Trump on Wednesday said that the United States will begin imposing 25% tariffs on goods imported from India on Aug. 1. 13 hours ago ... President Trump said Tuesday that India may be hit with a tariff rate of 20% to 25%. The country is one of the largest US trade partners ... 1 day ago ... On Friday, the United States is ready to impose a 26 percent tariff on all goods it imports from India. Dozens of countries are facing the same ... Ju

In [41]:
response['output']

'The United States has imposed tariffs on India in response to their trade practices, and India has retaliated with tariffs on American goods. This has led to tensions between the two nations and could potentially have a negative impact on their trade relationship.'